
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Conditional Tasks and Repair Runs

Databricks Workflow Jobs have the ability to run tasks based on the result of previously run tasks. For example, you can setup a task to only run if a previous task fails.

Also, when a task fails, you can repair the run and restart tasks without restarting the whole job. This can save a significant amount of time.

In this lesson, we will configure a pipeline with conditional logic, and we will learn how to repair runs. 

**Our goal is to create this job:**

![Lesson04_FullJobRun](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_FullJobRun.png)

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>
```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The **DA** object is only used in Databricks Academy courses and is not available outside of these courses.

In [0]:
%run ./Includes/Classroom-Setup-4

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## B. Create a Starter Job

1. Run the cell below to use the custom `DA` object to create a starter job for this demonstration. After the cell completes, it will create a job named **&lt;your-schema>_Lesson_04&gt;** with three individual tasks.

**NOTE:** The following custom method uses the Databricks SDK to programmatically create a job for demonstration purposes. You can find the method definition that uses the Databricks SDK to create the job in the [Classroom-Setup-Common]($./Includes/Classroom-Setup-Common) notebook. However, the [Databricks SDK](https://databricks-sdk-py.readthedocs.io/en/latest/) is outside the scope of this course.

In [0]:
DA.create_job_lesson04()

Created the job: labuser9084188_1738337208_Lesson_04
Job ID: 793978367435360


2. Open your new starter job **&lt;your-schema>_Lesson_04&gt;**:
  - a. Navigate to **Workflows** and open it in a new tab.
  
  - b. Select your new job, **&lt;your-schema>_Lesson_04&gt;**.
  
  - c. Select **Tasks** in the top navigation bar.
  
  - d. View your job. Notice that the job contains three tasks: **Ingest_Source_1**, **Ingest_Source_2**, and **Ingest_Source_3**. The three tasks should all be independent of one another. 

  - e. Leave the job open and return to these instructions.

  ![Lesson04_DefaultJob](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_DefaultJob.png)

## C. Explore the Task Notebooks
1. The following notebooks use simple code to demonstrate the functionality of creating workflows. Each notebook is in **Task Notebooks** > **Lesson 4 Notebooks**. You can use the links below to view each notebook used in the tasks and explore the code.

- Task: [Ingest_Source_1]($./Task Notebooks/Lesson 4 Notebooks/Ingest Source 1) notebook

- Task: [Ingest_Source_2]($./Task Notebooks/Lesson 4 Notebooks/Ingest Source 2) notebook

- Task: [Ingest_Source_3]($./Task Notebooks/Lesson 4 Notebooks/Ingest Source 3) notebook

## D. Set Dependencies on the Tasks
We are going to make some changes to this job and configure our main task to run only if all previous tasks succeed.

To simulate a real-world scenario, we will configure the job to either succeed or fail based on a task parameter. At this point, we want to observe the behavior when an upstream task fails, so we will configure the task parameter to intentionally cause the notebook to fail.

### D1. Add a Parameter to the Ingest_Source_1 Task

Configure the task parameter in the **Ingest_Source_1** task as follows:

1. Select the task **Ingest_Source_1**.

2. In the **Parameters** field, click **Add**.

3. For **Key**, type *test_value*.

4. For **Value**, type *Failure*.

    **NOTE:** Be aware values are case-sensitive

5. For **Retries**, select the edit button. In the **Retry Policy**, deselect *Enable serverless auto-optimization (may include at most 3 retries)*. This will turn off the retry policy if the job fails, preventing the task from running multiple times to save time during the demonstration.

6. Click **Confirm**.

5. Click **Save task**.

We now added a parameter to the **Ingest_Source_1** task. 

**The parameter value  *Failure* will cause an error when this notebook is run.**

### D2. Add the Clean_Data Task
Let's add the **Clean Data** notebook as a task named **Clean_Data**. The notebook will add dynamic value references to the task. 

Databricks Workflow Jobs provide [options](https://docs.databricks.com/en/workflows/jobs/parameter-value-references.html) for passing information about jobs/tasks to tasks and from one task to another. 


1. Open and explore the code in the [Clean Data]($./Task Notebooks/Lesson 4 Notebooks/Clean Data) notebook.

    **NOTE:** The notebook creates Dynamic value references using `dbutils.jobs.taskValues.set(key = 'bad_records', value = 5)`. This dynamically creates a **Key** named *bad_records* with the **Value** *5*. This can be set dynamically in whatever way you wish.


1. Close the **Clean Data** notebook.

1. Click **Add task**, and select **Notebook**. 

1. Name the task **Clean_Data**.

1. Click the path field, select the notebook, **Lesson 4 Notebooks/Clean Data**, and click **Confirm**

1. Click the **Depends on** field, and select all three of the tasks in the job (**Ingest_Source_1**, **Ingest_Source_2**, **Ingest_Source_3** ) to add them to the list. The DAG should show connections from the first three tasks to the **Clean_Data** task.

1. Click **All succeeded** in the **Run if dependencies** field to drop open the combo box. Note the variety of conditions available. 

1. Select **All succeeded**.

1. Click **Create task**.

![Lesson04_Clean_Data_Task](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_Clean_Data_Task.png)

### D3. Run the Job
1. Run the job by clicking **Run now** in the upper-right corner. 

2. A pop-up window appears with a link to the job run. Click **View run**.

3. Watch the tasks in the DAG. The colors change to show the progress of the task (about 2-3 minutes to complete):

  * **Gray** -- the task has not started
  * **Green stripes** -- the task is currently running
  * **Solid green** -- the task completed successfully
  * **Dark red** -- the task failed
  * **Light red** -- an upstream task failed, so the current task never ran

4. When the run is finished, note that **Ingest_Source_1** failed. This was expected. Also, note that our **Clean_Data** task never ran because we required that all three parent tasks must succeed before the **Clean_Data** task will run.

![Lesson04_Ingest_Source_1_Failure](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_Ingest_Source_1_Failure.png)

### D4. Repairing Job Runs

We have the ability to view the notebooks used in a task, including their output, as part of the job run. This can help us diagnose errors. We also have the ability to re-run specific tasks in a failed job run. 

Consider the following example:

You are developing a job that includes a handful of notebooks. During a run of the job, one of the tasks fails. You can change the code in that notebook and re-run that task and any tasks that depend on it. Additionally, you can change task parameters and re-run a task. Let's do this now:

1. In the upper-right corner, click **Repair run**. 

2. Make sure the **Ingest_Source_1** task is selected. Change the value for **test_value** from *Failure* to *Succeed*.

3. Highlight **Clean_Data** task so it is included in the Repair Run, then click **Repair run (2)** and let the job complete. You'll see Pipeline animate if a few seconds.

4. The "2" in the **Repair run** button indicates that Databricks selected both the failed task and the task that depended on it. You can select and deselect whichever tasks you wish to re-run.

    **NOTE:** When we use **Repair run** to change a parameter, the original parameter in the task definition is not changed—only the parameter in the current run is updated. In next Step, we will hard-code the Task parameter from *Failure* to *Succeed*.

5. After the repaired run is executed successfully complete the following:
    - a. Navigate back to the job by selecting it in the link above the job name
    
    - b. Select **Tasks** in the navigation bar.
    
    - c. Select the **Ingest_Source_1** task and modify the parameter value of **test_value** to **Succeed**
    
    - d. Then select **Save task**.


![Lesson04_RepairRun_01](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_RepairRun_01.png)




## E. Add If/Else Condition Task
We can also perform branching logic based on a boolean condition. Let's see an example of this in action. 

Suppose our **Clean_Data** task pushes bad records to a quarantine table. We will want to fix these bad records, if possible, before continuing to the next task. Let's set up this logic:

1. Go back to the job definition page for our job and make sure you are on the **Tasks** tab.

2. Select the **Clean_Data** task and click **Add task**. Then select **If/else condition** (under the **Control flow** section at the bottom of the list).

3. Name the task **Is_Record_Check_0**. Note have 2 new settings in node (*True* and *False*).

4. For the condition field complete the following:
    - a. Type `{{tasks.Clean_Data.values.bad_records}}` (make sure to include both sets of curly brace) in the left-hand side field.  

    - b. Choose **`==`** in the dropdown, and type *0* in the right-hand side field. This code obtains the value of the **bad_records** key in the **Clean_Data** task and compares it to the condition.

    **NOTE:** To obtain the task value with the key *value_name* that was set by task *task_name* use the following:`{{tasks.task_name.values.value_name}}`. View the [Supported value references](https://docs.databricks.com/en/jobs/dynamic-value-references.html#supported-value-references) for a list of dynamic value references that are supported.

5. Ensure **Depends on** is set to **Clean_Data** and **Run if dependencies** is set to **All succeeded**.

6. Click **Create task**.

![Lesson04_If_Condition](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_If_Condition.png)

### E2. Setting Our "False" Task
Let's setup a task that will fix bad records before moving on in the job. That is, if the **bad_records** value is not equal to *0*.:

1. Select the **Is_Record_Check_0** task and select **Add task** on the job definition page, then select **Notebook**.

1. Name the task **Fix_Bad_Records**.

1. For the path, navigate to **Lesson 4 Notebooks/Fix Bad Records**. 

    **NOTE:** The **Fix_Bad_Records** notebook will simply execute the code: `print("Logic that fixes bad records.")`. In a real world scenario you can set any logic here.

1. For **Depends on**, confirm only **Is_Record_Check_0 (false)**. 

    **NOTE:** Please make sure that **Is_Record_Check_0 (true)** is not selected!

1. Ensure that Run if dependencies is set to **All succeeded**.

1. Click **Create task**

We are setting this task to only run if the **Is_Record_Check_0** fails, meaning the value for the **bad_records** key did not equal 0. In our current scenario, the **Clean_Data** task sets the value of **bad_records** to *5*.


![Lesson04_Fix_Bad_Records_False](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_Fix_Bad_Records_False.png)

### E3. Aggregate Records Task
Let's setup a task for aggregating data. We want this task to either run if:
- If the **Is_Record_Check_0** is *true* (meaning `bad_records == 0`) or 
- If the **Is_Record_Check_0** is *false* (meaning `bad_records != 0`) and the **Fix_Bad_Records** task completed.

**NOTE:** The **Aggregate Records** notebook will simply run: `print("Aggregates records.")`. In a real world scenario you can add any logic you need.

1. Select the **Fix_Bad_Records** task then select **Add task** and select **Notebook**.

1. Name the task **Aggregate_Records**.

1. For the path, navigate to **Lesson 4 Notebooks/Aggregate Records**

1. For **Depends on**, select **Is_Record_Check_0 (true)** AND **Fix_Bad_Records**.

1. Set **Run if dependencies** to **At least one succeeded**.

1. Click **Create task**

This task will run if either:
- **Is_Record_Check_0** is *true*, (which is not the case since 5 does not equal 0) or
- **Fix_Bad_Records** completed successfully (this is the case for our example)

![Lesson04_FullJob](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_FullJob.png)

### E4. Run the Entire Job

##### IMPORTANT NOTE
Before running the job, go to the **Task** tab on the **Job Details** page and select **Ingest_Source_1**. In the **Parameters** section make sure the key **test_value** is set to *Succeed*.

1. Click **Run now** to run the entire job. Then in the pop up select **View job**.
  
2. At the moment, our hard-coded **bad_records** value is set to *5*, so we should see the **Fix_Bad_Records** task run before the **Aggregate_Records** task when we run the job.

![Lesson04_FullJobRun](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_FullJobRun.png)

## Bonus lab: 
###Change *Is_Record_Check_0* task to fire a *True* condition

1. From *Tasks* tab, open the  **Is_Record_Check_0** task.

2. In **Condition** caption change 0 to 5.  Now 5 (from **Clean_Data** task) = 5 (in this task). 

3. For completeness, change the *Task name* to **Is_Record_Check_5**.  This will fire the **True** condition when run and flow through to the **Aggregate_Records** task.  Click on **Run now** button to confirm.


##### Final Run
![Lesson04_True_run](files/images/deploy-workloads-with-databricks-workflows-2.0.1/Lesson04_True_run.png)


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>